In [1]:
import numpy
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [2]:
from scipy.spatial.distance import euclidean
import os
import numpy as np
def load_speaker_mfccs(data_dir):
    speakers = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    speaker_model = {}

    for speaker in speakers:
        speaker_dir = os.path.join(data_dir, speaker)
        mfcc_features = []

        for root, _, files in os.walk(speaker_dir):
            for file_name in files:
                if file_name.endswith('.flac'):
                    file_path = os.path.join(root, file_name)
                    if os.path.exists(file_path):
                        try:
                            y, sr = librosa.load(file_path, sr=None)
                            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
                            mfcc_features.append(mfccs.T)
                        except Exception as e:
                            print(f"Error processing file {file_path}: {e}")

        if mfcc_features:
            speaker_model[speaker] = np.vstack(mfcc_features)

    return speaker_model

In [3]:
def prediction(test_files, speaker_model, true_speaker):
    correct_predictions = 0
    for file_path in test_files:
        predicted_speaker = identify_speaker(file_path, speaker_model)
        print(f"Predicted: {predicted_speaker}, True: {true_speaker}")
        if predicted_speaker == true_speaker:
            correct_predictions += 1

    accuracy = correct_predictions / len(test_files)
    return accuracy

In [ ]:
data_dir = "C:\\Users\\NIGHIL NATARAJAN\\Downloads\\DM-audio-files\\DM-audio-files"
speaker_model = load_speaker_mfccs(data_dir)

test1 = {
    'Raghav': ["C:\\Users\\NIGHIL NATARAJAN\\Downloads\\DM-audio-files\\DM-audio-files\\Raghav\\Raghav15.flac",
               "C:\\Users\\NIGHIL NATARAJAN\\Downloads\\DM-audio-files\\DM-audio-files\\Raghav\\Raghav24.flac"],
    'Nighil': ["C:\\Users\\NIGHIL NATARAJAN\\Downloads\\DM-audio-files\\DM-audio-files\\Nighil\\Nighil9.flac",
               "C:\\Users\\NIGHIL NATARAJAN\\Downloads\\DM-audio-files\\DM-audio-files\\Nighil\\Nighil8.flac"],
    'Sathya': ["C:\\Users\\NIGHIL NATARAJAN\\Downloads\\DM-audio-files\\DM-audio-files\\Sathya\\Sathya55.flac",
               "C:\\Users\\NIGHIL NATARAJAN\\Downloads\\DM-audio-files\\DM-audio-files\\Sathya\\Sathya71.flac"],
    'Sandeep': ["C:\\Users\\NIGHIL NATARAJAN\\Downloads\\DM-audio-files\\DM-audio-files\\Sandeep\\Sandeep33.flac",
                "C:\\Users\\NIGHIL NATARAJAN\\Downloads\\DM-audio-files\\DM-audio-files\\Sandeep\\Sandeep43.flac"],
    'Subrajith': ["C:\\Users\\NIGHIL NATARAJAN\\Downloads\\DM-audio-files\\DM-audio-files\\Subrajith\\Subrajith33.flac",
                  "C:\\Users\\NIGHIL NATARAJAN\\Downloads\\DM-audio-files\\DM-audio-files\\Subrajith\\Subrajith43.flac"]
}

C:\Users\NIGHIL NATARAJAN\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [5]:
def identify_speaker(test_file_path, speaker_model):
    try:
        y, sr = librosa.load(test_file_path, sr=None)
        test_mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T
        test_mean = np.mean(test_mfcc, axis=0)

        min_dist = float('inf')
        predicted_speaker = None

        for speaker, mfccs in speaker_model.items():
            speaker_mean = np.mean(mfccs, axis=0)
            dist = euclidean(test_mean, speaker_mean)

            if dist < min_dist:
                min_dist = dist
                predicted_speaker = speaker

        return predicted_speaker

    except Exception as e:
        print(f"Error identifying speaker: {e}")
        return None

# STEP 5: Evaluate accuracy per speaker
for speaker, files in test1.items():
    accuracy = prediction(files, speaker_model, speaker)
    print(f'Accuracy for {speaker}: {accuracy:.2f}\n')

Predicted: Raghav, True: Raghav
Predicted: Raghav, True: Raghav
Accuracy for Raghav: 1.00

Predicted: Nighil, True: Nighil
Predicted: Nighil, True: Nighil
Accuracy for Nighil: 1.00

Predicted: Sathya, True: Sathya
Predicted: Sathya, True: Sathya
Accuracy for Sathya: 1.00

Predicted: Sathya, True: Sandeep
Predicted: Sathya, True: Sandeep
Accuracy for Sandeep: 0.00

Predicted: Sathya, True: Subrajith
Predicted: Subrajith, True: Subrajith
Accuracy for Subrajith: 0.50

